In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
train = pd.read_csv('./filedown (2)/train.csv')
test = pd.read_csv('./filedown (2)/public_test.csv')

In [3]:
train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [4]:
train.shape

(295945, 4)

In [5]:
train.smishing.value_counts()

0    277242
1     18703
Name: smishing, dtype: int64

# spall checking by pusan
- 시간 상의 문제로 적용 X

In [6]:
import re
text = train.text.map(lambda x : re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', x))

In [7]:
from chatspace import ChatSpace

spacer = ChatSpace()

Loading JIT Compiled ChatSpace Model


In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
from copy import copy
import time

def spallcheck_using_pusan(reviews, num=10):
    res, is_transform, error = [], [], False
    def spallcheck(driver, text):
        checked_text = copy(text)
        driver.find_element_by_name('text1').send_keys(text)
        driver.find_element_by_id('btnCheck').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        is_valid = soup.find('div', 
                 {"class":"divScrollbarStyle", 'id':'divLeft1'})
        if is_valid:
            announceText = soup.find('td', 
                         {'id':'pageAnnounce'}).text
            t = is_valid.td
            rule = {f.next.next_sibling : f.span.text for f in t.find_all('font')}
            if announceText:
                for i in range(1, int(announceText.split(' ')[1].split('페이지')[0])):
                    driver.find_element_by_id('nextBtn').click()
                    t = soup.find('div', {"class":"divScrollbarStyle", 'id':'divLeft1'}).td
                    for f in t.find_all('font'):
                        rule[f.next.next_sibling] = f.span.text
            for i,j in rule.items():
                checked_text = checked_text.replace(i, j)     
            transform = True
        else:
            transform = False
        driver.find_element_by_id('btnRenew2').click()
        return checked_text, transform
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    # 혹은 options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    options.add_argument("lang=ko_KR") # 한국어!
    driver = webdriver.Chrome(
        'C:/chromedriver.exe', options=options)
    print('---** selenium으로 web에 접근 중 ... 3초 소요됩니다... **---')
    driver.implicitly_wait(3)
    print('Connected. spaller.cs.pusan.ac.kr/에 접속합니다.')
    driver.get('about:blank')
    driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5];},});")
    driver.get('https://speller.cs.pusan.ac.kr/')
    print('text 분석을 실시합니다. {}개 text마다 결과를 기록합니다.'.format(num))
    start, semi_start = time.time(), time.time()
    for ix, text in enumerate(reviews):
        if ix % num == 0:
            if time.time() - start >= 60:
                a = (time.time() - start) / 3600
                b = 'hours'
            else:
                a = (time.time() - start) / 60
                b = 'mins'
            print(ix, '\t\tcollapse {:.2f} mins\t\tcumulative time is {:.2f} {:s}'.format(
                (time.time() - semi_start) / 60, a, b))
            semi_start = time.time()
        try:
            checked_text, transform = spallcheck(driver, text)
        except:
            print('Error: 에러 발생!')
            error = True
            return res, is_transform, error
        res.append(checked_text)
        is_transform.append(transform)
    driver.close()
    return res, is_transform, error

In [20]:
res, is_transform, error = spallcheck_using_pusan(
    train['text'].values, 100)

---** selenium으로 web에 접근 중 ... 3초 소요됩니다... **---
Connected. spaller.cs.pusan.ac.kr/에 접속합니다.
text 분석을 실시합니다. 100개 text마다 결과를 기록합니다.
0 		collapse 0.00 mins		cumulative time is 0.00 mins
100 		collapse 4.15 mins		cumulative time is 0.07 hours
Error: 에러 발생!


In [21]:
len(res), len(is_transform)

(168, 168)

In [22]:
res[-1]

'XXX 고객님 XXX 은행 성우금융센터지점 VIP 팀장 XXX입니다. 저는 이번에 송우 종합금융 센터 지점을 떠나 만X XX 점으로 이동하게 되었습니다. 항상 성우금융센터지점을 사랑해 주신 고객님께 깊이 감사드립니다. 이별은 만남의 시작이라는 말이 있듯이 언젠가는 꼭 또 다른 모습으로 만날 것이라 기대하며 그 만남을 위해 열심히 나아가겠습니다. 이렇게 문자를 통해 인사드리게 된 점 양해해 주시기 바 XXX 고객님의 가정에 항상 건강과 행운이 함께하기를 기원하겠습니다.XXX 은행송우 종합금융 센터XXX올림'

In [23]:
train.loc[247, 'text']

'XXX 고객님안녕하세요. XXX은행 XXX지점 XXX입니다. 제가 인사이동으로 16일부터 양재역 종합금융센터에서 근무하게 되었습니다.    XXX지점에서 근무하면서 생겼던 일들을 생각하면 아련하게 추억이 될 것을 생각하니 아쉽고 서운한 마음이 가득합니다.  찾아뵙고 인사를 드려야 하나 형편상 그러지 못한 점 널리 양해부탁드립니다.  저에 대한 좋은 기억을 갖어 주시길 부탁드립니다. 항상 건강하시고 가정에 축복이 많으시길 기원합니다.'

# Baseline

In [24]:
train[train['smishing'] == 1]['text'].map(
    lambda x : '(광고)' in x).sum() / train[train['smishing'] == 1].shape[0]

0.9310271079505962

In [25]:
train[train['smishing'] != 1]['text'].map(
    lambda x : '(광고)' in x).sum() / train[train['smishing'] != 1].shape[0]

0.05850123718628491

#### naive hypothesis
(광고)라는 단어가 들어가면 smishing일 것이다

In [28]:
submission = pd.DataFrame(test['id'])
submission['smishing'] = test['text'].map(lambda x : 1 if '(광고)' in x else .05)
submission.to_csv('submission_baseline.csv', index=False)

In [34]:
submission = pd.read_csv('submission_제출양식.csv')
submission['smishing'] = test['text'].map(
    lambda x : min(1, np.random.normal(99, 0.1)) 
    if '(광고)' in x 
    else max(0, np.random.normal(0.05, 0.03))).values
submission.to_csv('submission_baseline2.csv', index=False)

결과는 81%

# New

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf = TfidfVectorizer()
cvect = CountVectorizer()

In [36]:
train.smishing.value_counts()

0    277242
1     18703
Name: smishing, dtype: int64

In [37]:
df = pd.concat((train[train.smishing == 0].sample(18000), 
                train[train.smishing == 1].sample(18000)))

In [40]:
df.smishing.value_counts()

1    18000
0    18000
Name: smishing, dtype: int64

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train.text, train.smishing, shuffle=True, 
    stratify=train.smishing)

In [42]:
X_train = cvect.fit_transform(X_train)
X_test = cvect.transform(X_test)

In [43]:
from sklearn.naive_bayes import MultinomialNB

gnb_clf = MultinomialNB()

In [44]:
gnb_clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [45]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

print(accuracy_score(y_train.values, gnb_clf.predict(X_train)))
print(f1_score(y_train.values, gnb_clf.predict(X_train)))
print(roc_auc_score(y_train.values, gnb_clf.predict(X_train)))

0.9997386893015796
0.9979332953249715
0.9990627491313326


In [46]:
print(accuracy_score(y_test.values, gnb_clf.predict(X_test)))
print(f1_score(y_test.values, gnb_clf.predict(X_test)))
print(roc_auc_score(y_test.values, gnb_clf.predict(X_test)))

0.9997161663535485
0.9977547311023202
0.9988513575248215


In [47]:
testX = cvect.transform(test.text)

In [48]:
submission['smishing'] = gnb_clf.predict_proba(testX)[:,1]
submission.to_csv('tfidf_gnb5.csv', index=False)

In [49]:
submission.head()

,id,smishing
0,340000,3.321947e-124
1,340001,1.852452e-56
2,340002,3.652442e-24
3,340003,2.325928e-16
4,340004,1.593863e-46


결과는 82.744%